In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import urllib, json
from PIL import Image
import datetime
import openpyxl

In [ ]:
# Подключаем api_key из файла
f = open('API_KEY.txt', 'r')
api_key = f.read()
f.close()

In [ ]:
# Составляем URL request
api_root = 'https://api.nasa.gov/'
api_product = 'neo/rest/v1/feed?'
start_date = '2021-01-01'
end_date = '2021-01-01'
url = f"{api_root}{api_product}start_date={start_date}&end_date={end_date}&api_key={api_key}"

In [ ]:
'''
Функция навигации по данным из api response   
возвращает ссылку на данные в другом временном отрезке

    step = 'next' -- следующий день от заданного start_date в URL request
    step = 'prev' -- предыдущий день от заданного start_date в URL request
    step = 'self' -- текущий день, равный start_date в URL request
'''

def get_url(url, step):
    url_level = urllib.request.urlopen(url)
    data_level = json.loads(url_level.read())['links']
    if step == 'next':
        return data_level[step]
    elif step == 'prev':
        return data_level[step]
    elif step == 'self':
        return data_level[step]

In [ ]:
'''
Вариант использования бибилотеки datetime для скольжения по датам (не используется в коде)
и вывод даты в строку

delta = datetime.timedelta(days=1)
day_current = datetime.date.fromisoformat(start_date) + delta
date_str = datetime.date.strftime(day_current, '%Y-%m-%d')
'''

In [ ]:
'''Навигация по ссылкам.

Работаем с данными за один день 2020-07-07.
Обрабатываем входящий json.
'''

url_level_0 = urllib.request.urlopen(url)
data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
data_level_0

In [ ]:
# Выбираем один элемент из списка за один день
data_day = data_level_0.near_earth_objects[3]
data_day_item = data_day[0]
data_day_item

In [ ]:
# Создаем словари с необходимями данными для DataFrame
dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
dict_close_approach_data = data_day_item['close_approach_data'][0]
dict_relative_velocity = dict_close_approach_data['relative_velocity']
dict_miss_distance = dict_close_approach_data['miss_distance']

In [ ]:
# Чистим словарь от вложенных словарей
keys = ['links','estimated_diameter','close_approach_data', 'relative_velocity', 'miss_distance']

for key in keys:
    data_day_item.pop(key, None)
data_day_item_short = data_day_item

data_day_item_short

In [ ]:
'''Создаем DataFrame для выбранных данных.

Объединяем DataFrame в один для дальнейшей работы.
'''

df1 = pd.DataFrame([data_day_item_short])
df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)
df_item = df1.join(df2).join(df3).join(df4).join(df5)

df_item

In [ ]:
# Чистим DataFrame от нерелевантных данных, переименовываем
df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                'neo_reference_id',
                                                'miss_distance',
                                                'miles_per_hour',
                                                'astronomical',
                                                'lunar',
                                                'miles'
                                                ], axis=1
).rename(columns={'kilometers': 'miss_distance_kilometers',
                'estimated_diameter_min': 'estimated_diameter_km_min',
                'estimated_diameter_max': 'estimated_diameter_km_max'})

# Создаем словарь из DataFrame
df_item.to_dict()

**Создаем цикл для работы с API с получением данных за определенное количество дней**

In [ ]:
'''Структуры для работы.

items_list - список из данных по каждому объекту за определенный период
missed_colums_keys = [] - список для обработки словаря по одному объекту (удаление столбцов)
'''
items_list = []
missed_colums_keys = [
    'links','estimated_diameter',
    'close_approach_data',
    'relative_velocity',
    'miss_distance'
    ]

In [ ]:
'''Создание списка заголовков

для требуемого пустого DataFrame
из DataFrame за один день
'''
key_list = list(df_item.to_dict())
key_list.remove('epoch_date_close_approach')
key_list.remove('relative_velocity')
key_list

In [ ]:
# Создаем пустой DataFrame
df0 = pd.DataFrame(columns=key_list)

df0

In [ ]:
# n - количество запросов(дней)
n = 365

In [ ]:
''' Создаем цикл для получения данных

через API за заданный период времени
на основе алгоритма получения данных
за один день
'''


for i in range(1, n+1):
    current = get_url(url, 'next')
    url = current
    
    # Навигация по ссылкам, работаем с данными за один день
    url_level_0 = urllib.request.urlopen(url)
    data_level_0 = pd.DataFrame.from_dict(json.loads(url_level_0.read()))
    
    # Cписок элементов за один день
    data_in_day = data_level_0.near_earth_objects[3]

    # Цикл по элементам списка за один день
    for data_day_item in data_in_day:
        dict_estimated_diameter = data_day_item['estimated_diameter']['kilometers']
        dict_close_approach_data = data_day_item['close_approach_data'][0]
        dict_relative_velocity = dict_close_approach_data['relative_velocity']
        dict_miss_distance = dict_close_approach_data['miss_distance']

        # Чистим словарь от вложенных словарей
        for key in missed_colums_keys:
            data_day_item.pop(key, None)

        data_day_item_short = data_day_item
        
        # Создаем Dataframe из вложенных словарей
        df1 = pd.DataFrame([data_day_item_short])
        df2 = pd.DataFrame(dict_estimated_diameter, index=[0]).reset_index(drop=True)
        df3 = pd.DataFrame(dict_close_approach_data, index=[0]).reset_index(drop=True)
        df4 = pd.DataFrame(dict_relative_velocity, index=[0]).reset_index(drop=True)
        df5 = pd.DataFrame(dict_miss_distance, index=[0]).reset_index(drop=True)
        
        # Чистим DataFrame от нерелевантных данных, переименовываем
        df_item = df1.join(df2).join(df3).join(df4).join(df5).drop(['nasa_jpl_url',
                                                        'neo_reference_id',
                                                        'miss_distance',
                                                        'miles_per_hour',
                                                        'astronomical',
                                                        'lunar',
                                                        'miles',
                                                        'epoch_date_close_approach',
                                                        'relative_velocity'
                                                        ], axis=1
        ).rename(columns={'kilometers': 'miss_distance_kilometers',
                        'estimated_diameter_min': 'estimated_diameter_km_min',
                        'estimated_diameter_max': 'estimated_diameter_km_max'})
        

        # Добавление в итоговый DataFrame данных построчно
        df0 = pd.concat([df0, df_item])

In [ ]:
df0.dtypes

In [ ]:
# Меняем тип данных, округляем float
df0['id'] = df0['id'].astype(float).astype(int)
df0['is_potentially_hazardous_asteroid'] = (df0['is_potentially_hazardous_asteroid']*1).astype(int)
df0['is_sentry_object'] = (df0['is_sentry_object']*1).astype(int)
df0['absolute_magnitude_h'] = df0['absolute_magnitude_h'].astype('float64').round(2)
df0['kilometers_per_second'] = df0['kilometers_per_second'].astype('float64').round(3)
df0['estimated_diameter_km_min'] = df0['estimated_diameter_km_min'].astype('float32').round(3)
df0['estimated_diameter_km_max'] = df0['estimated_diameter_km_max'].astype('float32').round(3)
df0['close_approach_date'] = pd.to_datetime(df0['close_approach_date'])
df0['close_approach_date_full'] = pd.to_datetime(df0['close_approach_date_full'])
df0['kilometers_per_hour'] = df0['kilometers_per_hour'].astype(float).astype(int)
df0['miss_distance_kilometers'] = df0['miss_distance_kilometers'].astype(float).astype(int)
df0.dtypes

In [ ]:
df0

In [ ]:
#df_from_xlsx = pd.read_excel("NASA_Asteroids_2022.xlsx",header=0)
#df_sum = pd.concat([df_from_xlsx, df0])
#df0.to_excel("NASA_Asteroids_2021.xlsx",
#             sheet_name='Sheet_1')

In [ ]:
#aaa = df_from_xlsx.drop(['Unnamed: 0'], axis=1)
#aaa

In [ ]:
#df0.dtypes

In [ ]:
#df0.to_csv('NASA_Asteroids_2022.csv')